### simple fio config file
```[global]
rw=readwrite
size=${FILE_SIZE}
thread=1
directory=${FIO_DIR}

[readwrite-iotest]
ioengine=${IOENGINE}
```

### Entrypoint
>fio [config file]

### image
>kai10k/alpine-fio

In [1]:
from k8sclient.keywords import (
    list_ready_nodes,
    wait_for_pod_state,
    SUCCEEDED,
    tail_pod_logs,
    delete_pod,
    NOT_FOUND
)
from k8sclient.Components import (
    PodBuilder,
    HostPathVolume,
    RBDVolume,
    CephFSVolume,
    EmptyDirVolume
)

image = "127.0.0.1:30100/library/alpine-fio:v1"
args = "--output-format=json"
namespace = "monkey"
nodes = list_ready_nodes()
FIO_DIR = "/mnt/fio"
ceph_monitors = "10.19.137.144:6789,10.19.137.145:6789,10.19.137.146:6789"
ceph_pool = "monkey"
ceph_fstype = "xfs"
ceph_secret = "ceph-secret"  # need create beforehand
# must match the regex [a-z0-9]([-a-z0-9]*[a-z0-9])?
empty_dir = EmptyDirVolume("empty-dir-fio", FIO_DIR)
ceph_fs = CephFSVolume(
            "cephfs",
            FIO_DIR,
            monitors=ceph_monitors,
            secret_name=ceph_secret,
            fs_path="/",
            sub_path="monkey-fio"
        )
rbd = RBDVolume(
        "rbd",
        FIO_DIR,
        fs_type=ceph_fstype,
        image="default",
        pool=ceph_pool,
        monitors=ceph_monitors,
        secret_name=ceph_secret,
        sub_path="fio",
    )

volumes = {
    "empty_dir": empty_dir,
    "rbd": rbd,
    "ceph_fs": ceph_fs
}
io_engines = ["libaio", "mmap", "posixaio", "sync"]


def test(node):
    print node
    pod_name = "fio-" + "-".join(node.split("."))
    reports = []
    for n, v in volumes.items():
        print n
        for e in io_engines:
            print e
            PodBuilder(
                pod_name,
                namespace,
            ).set_node(
                node
            ).add_container(
                pod_name + "-container",
                image=image,
                args=args,
                limits={'cpu': '1', 'memory': '512Mi'},
                requests={'cpu': '0', 'memory': '0'},
                volumes=[v],
                FIO_DIR=FIO_DIR,
                IOENGINE=e
            ).deploy()
            # wait to complete
            wait_for_pod_state(namespace, pod_name, timeout=3600, expect_status=SUCCEEDED)
            logs = tail_pod_logs(namespace, pod_name).strip()
            # delete the pod
            delete_pod(namespace, pod_name)
            wait_for_pod_state(namespace, pod_name, timeout=240, expect_status=NOT_FOUND)
            # report = json.loads(logs)
            report = eval(logs)
            print report
            for job in report["jobs"]:
                print "READ:", job['read']['bw'], "KB/s"
                print "WRITE:", job['write']['bw'], "KB/s"
            reports.append({
                "vtype": n,
                "io_engine": e,
                "read(MB/s)": float(report["jobs"][0]['read']['bw'])/1024,
                "write(MB/S)": float(report["jobs"][0]['write']['bw'])/1024
            })
    return reports

In [2]:
disk_io_512M = test("10.19.137.150")

10.19.137.150
ceph_fs
libaio
{u'jobs': [{u'job options': {u'ioengine': u'libaio'}, u'trim': {u'io_bytes': 0, u'slat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'drop_ios': 0, u'lat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'iops': 0.0, u'short_ios': 0, u'bw_mean': 0.0, u'bw_min': 0, u'total_ios': 0, u'bw_max': 0, u'bw': 0, u'bw_agg': 0.0, u'clat': {u'max': 0, u'percentile': {u'70.000000': 0, u'5.000000': 0, u'50.000000': 0, u'99.990000': 0, u'30.000000': 0, u'10.000000': 0, u'60.000000': 0, u'0.00': 0, u'1.000000': 0, u'90.000000': 0, u'99.950000': 0, u'99.000000': 0, u'40.000000': 0, u'20.000000': 0, u'99.900000': 0, u'95.000000': 0, u'99.500000': 0, u'80.000000': 0}, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'runtime': 0, u'bw_dev': 0.0}, u'jobname': u'readwrite-iotest', u'latency_target': 0, u'latency_ms': {u'750': 0.0, u'>=2000': 0.0, u'250': 0.0, u'100': 0.0, u'10': 0.0, u'20': 0.01, u'50': 0.0, u'2000': 0.0, u'2': 0.0, u'4': 0.0, u'500': 0.0, u'1000':

{u'jobs': [{u'job options': {u'ioengine': u'sync'}, u'trim': {u'io_bytes': 0, u'slat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'drop_ios': 0, u'lat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'iops': 0.0, u'short_ios': 0, u'bw_mean': 0.0, u'bw_min': 0, u'total_ios': 0, u'bw_max': 0, u'bw': 0, u'bw_agg': 0.0, u'clat': {u'max': 0, u'percentile': {u'70.000000': 0, u'5.000000': 0, u'50.000000': 0, u'99.990000': 0, u'30.000000': 0, u'10.000000': 0, u'60.000000': 0, u'0.00': 0, u'1.000000': 0, u'90.000000': 0, u'99.950000': 0, u'99.000000': 0, u'40.000000': 0, u'20.000000': 0, u'99.900000': 0, u'95.000000': 0, u'99.500000': 0, u'80.000000': 0}, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'runtime': 0, u'bw_dev': 0.0}, u'jobname': u'readwrite-iotest', u'latency_target': 0, u'latency_ms': {u'750': 0.01, u'>=2000': 0.01, u'250': 0.01, u'100': 0.01, u'10': 0.01, u'20': 0.01, u'50': 0.01, u'2000': 0.01, u'2': 0.01, u'4': 0.01, u'500': 0.01, u'1000': 0.01}, u'read': {u'i

{u'jobs': [{u'job options': {u'ioengine': u'posixaio'}, u'trim': {u'io_bytes': 0, u'slat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'drop_ios': 0, u'lat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'iops': 0.0, u'short_ios': 0, u'bw_mean': 0.0, u'bw_min': 0, u'total_ios': 0, u'bw_max': 0, u'bw': 0, u'bw_agg': 0.0, u'clat': {u'max': 0, u'percentile': {u'70.000000': 0, u'5.000000': 0, u'50.000000': 0, u'99.990000': 0, u'30.000000': 0, u'10.000000': 0, u'60.000000': 0, u'0.00': 0, u'1.000000': 0, u'90.000000': 0, u'99.950000': 0, u'99.000000': 0, u'40.000000': 0, u'20.000000': 0, u'99.900000': 0, u'95.000000': 0, u'99.500000': 0, u'80.000000': 0}, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'runtime': 0, u'bw_dev': 0.0}, u'jobname': u'readwrite-iotest', u'latency_target': 0, u'latency_ms': {u'750': 0.01, u'>=2000': 0.0, u'250': 0.015354, u'100': 0.045729, u'10': 0.034046, u'20': 0.01, u'50': 0.01874, u'2000': 0.0, u'2': 0.034523, u'4': 0.059795, u'500': 0.01, u'10

{u'jobs': [{u'job options': {u'ioengine': u'mmap'}, u'trim': {u'io_bytes': 0, u'slat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'drop_ios': 0, u'lat': {u'max': 0, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'iops': 0.0, u'short_ios': 0, u'bw_mean': 0.0, u'bw_min': 0, u'total_ios': 0, u'bw_max': 0, u'bw': 0, u'bw_agg': 0.0, u'clat': {u'max': 0, u'percentile': {u'70.000000': 0, u'5.000000': 0, u'50.000000': 0, u'99.990000': 0, u'30.000000': 0, u'10.000000': 0, u'60.000000': 0, u'0.00': 0, u'1.000000': 0, u'90.000000': 0, u'99.950000': 0, u'99.000000': 0, u'40.000000': 0, u'20.000000': 0, u'99.900000': 0, u'95.000000': 0, u'99.500000': 0, u'80.000000': 0}, u'min': 0, u'stddev': 0.0, u'mean': 0.0}, u'runtime': 0, u'bw_dev': 0.0}, u'jobname': u'readwrite-iotest', u'latency_target': 0, u'latency_ms': {u'750': 0.01, u'>=2000': 0.01, u'250': 0.012827, u'100': 0.018883, u'10': 0.111485, u'20': 0.037575, u'50': 0.031519, u'2000': 0.01, u'2': 0.253916, u'4': 0.295448, u'500': 0.01, u

In [3]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df2 = pd.DataFrame(disk_io_512M)
df2.set_index(["vtype", "io_engine"])

read(MB/s)  write(MB/S)
vtype     io_engine                         
ceph_fs   libaio          42.83        42.84
          mmap            39.07        39.08
          posixaio        47.30        47.31
          sync            57.27        57.29
empty_dir libaio          48.87        48.89
          mmap            22.13        22.14
          posixaio        21.16        21.17
          sync            50.32        50.34
rbd       libaio          30.34        30.35
          mmap            19.61        19.62
          posixaio        15.18        15.18
          sync            46.51        46.53

In [67]:
df.set_index(["vtype", "io_engine"]) #140 8G file 1G ram

read(MB/s)  write(MB/S)
vtype     io_engine                         
ceph_fs   libaio          74.80        74.82
          mmap            39.75        39.77
          posixaio        34.52        34.54
          sync            70.58        70.60
empty_dir libaio         179.29       179.35
          mmap           118.43       118.47
          posixaio        73.62        73.64
          sync           167.58       167.64
rbd       libaio          22.49        22.50
          mmap            18.54        18.54
          posixaio         7.22         7.22
          sync            23.89        23.90

In [71]:
paas_ext4 = pd.DataFrame([
    {"vtype": "rbd/ext4", "io_engine": "libaio", "read": "43.3MB/s", "write": "43.4MB/s"},
    {"vtype": "rbd/ext4", "io_engine": "mmap", "read": "45.4MB/s", "write": "45.5MB/s"},
    {"vtype": "rbd/ext4", "io_engine": "posixaio", "read": "4465kB/s", "write": "4466kB/s"},
    {"vtype": "rbd/ext4", "io_engine": "sync", "read": "48.8MB/s", "write": "48.8MB/s"},
])
# paas_ext4.set_index(["vtype", "io_engine"])

paas_xfs = pd.DataFrame([
    {"vtype": "rbd/xfs", "io_engine": "libaio", "read": "40.5MB/s", "write": "40.5MB/s"},
    {"vtype": "rbd/xfs", "io_engine": "mmap", "read": "43.3MB/s", "write": "43.3MB/s"},
    {"vtype": "rbd/xfs", "io_engine": "posixaio", "read": "4381kB/s", "write": "4383kB/s"},
    {"vtype": "rbd/xfs", "io_engine": "sync", "read": "46.2MB/s", "write": "46.2MB/s"},
])
paas_xfs.set_index(["vtype", "io_engine"])

read     write
vtype   io_engine                    
rbd/xfs libaio     40.5MB/s  40.5MB/s
        mmap       43.3MB/s  43.3MB/s
        posixaio   4381kB/s  4383kB/s
        sync       46.2MB/s  46.2MB/s